<a href="https://colab.research.google.com/github/Noahkandie/Covid-19-Tweets-sentiment-analysis/blob/Models/Covid_19_Vaccine_Tweets_sentiment_Transformers_Distilbert_Uncased_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing Libraries

In [1]:
import numpy as np    
import pandas as pd 
import matplotlib.pyplot as plt
import tensorflow as tf


In [2]:
! pip install transformers

In [3]:
import transformers
from transformers import DistilBertTokenizer
from transformers import TFDistilBertForSequenceClassification

In [4]:
! pip install pyyaml h5py

In [5]:
!pip install wandb
!wandb login 
import wandb
from wandb.keras import WandbCallback

wandb: Currently logged in as: noahkandie (use `wandb login --relogin` to force relogin)


232cc0cd456e67d2107c6250b358804929b7b73a

In [6]:
!pip3 install ktrain

In [7]:
#initialise wandb
#%env WANDB_PROJECT='gpt3', entity='alpha-coders'
#%env WANDB_LOG_MODEL=true

In [8]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import os
os.environ["CUDA_DEVICE_ORDER"]="PCI_BUS_ID";
os.environ["CUDA_VISIBLE_DEVICES"]="0";

In [9]:
import ktrain
from ktrain import text
import pandas as pd

In [10]:
data_df = pd.read_csv('/content/Covid-19 vaccine tweets sentiments.csv')


In [11]:
data_df['label']=data_df['sentiment'].copy()

In [12]:
data_df.head()

,Unnamed: 0,tweet,location,created_at,vaccine,town,country,date,year,month,day,hour,minute,clean_text,week,scores,compound,sentiment,sentiment2,label
0,0,2020 BioNTech &amp; Pfizer announce their COVI...,"Nairobi, Kenya",2021-11-09 06:41:56,Pfizer-BioNTech,Nairobi,Kenya,2021-11-09,2021.0,11.0,9,6,41.0,biontech amp pfizer announce covid vaccine eff...,Week 1,"{'neg': 0.0, 'neu': 0.763, 'pos': 0.237, 'comp...",0.4767,positive,positive,positive
1,1,@NamanyaRacheal2 @UgandaBreweries @centurybott...,"Kampala, Uganda",2021-11-09 05:45:27,Pfizer-BioNTech,Kampala,Uganda,2021-11-09,2021.0,11.0,9,5,45.0,namanyaracheal ugandabreweries centurybottling...,Week 1,"{'neg': 0.0, 'neu': 0.777, 'pos': 0.223, 'comp...",0.3182,positive,positive,positive
2,2,.@MinofHealthUG received a total of 3.4 millio...,"Kampala, Uganda",2021-11-09 05:20:05,Pfizer-BioNTech,Kampala,Uganda,2021-11-09,2021.0,11.0,9,5,20.0,minofhealthug receive total million dose covid...,Week 1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral,neutral,neutral
3,3,@Case18640259 I think pfizer is the worst,"Kampala, Uganda",2021-11-09 05:09:20,Pfizer-BioNTech,Kampala,Uganda,2021-11-09,2021.0,11.0,9,5,9.0,case think pfizer worst,Week 1,"{'neg': 0.577, 'neu': 0.423, 'pos': 0.0, 'comp...",-0.6249,negative,strongly negative,negative
4,4,On Sunday we got more vaccines from @usmission...,soroti,2021-11-09 03:43:24,Pfizer-BioNTech,soroti,Uganda,2021-11-09,2021.0,11.0,9,3,43.0,sunday get vaccine usmissionuganda abt million...,Week 1,"{'neg': 0.0, 'neu': 1.0, 'pos': 0.0, 'compound...",0.0000,neutral,neutral,neutral


In [13]:
data_df['label'].value_counts()

neutral     5532
positive    4484
negative    2675
Name: label, dtype: int64

In [14]:
data_df.isnull().sum()

Unnamed: 0      0
tweet           0
location      155
created_at      0
vaccine         0
town          155
country         0
date            0
year            0
month           0
day             0
hour            0
minute          0
clean_text      0
week            0
scores          0
compound        0
sentiment       0
sentiment2      0
label           0
dtype: int64

In [15]:
data_df.dropna(inplace=True)

In [16]:
data_df.isnull().sum()

Unnamed: 0    0
tweet         0
location      0
created_at    0
vaccine       0
town          0
country       0
date          0
year          0
month         0
day           0
hour          0
minute        0
clean_text    0
week          0
scores        0
compound      0
sentiment     0
sentiment2    0
label         0
dtype: int64

In [17]:
# creating a dataframe with the tweet and target column
data_df= data_df[['clean_text','label']]
data_df.head()

,clean_text,label
0,biontech amp pfizer announce covid vaccine eff...,positive
1,namanyaracheal ugandabreweries centurybottling...,positive
2,minofhealthug receive total million dose covid...,neutral
3,case think pfizer worst,negative
4,sunday get vaccine usmissionuganda abt million...,neutral


In [18]:
# Defining our X and Y varaibles
X = data_df['clean_text']
y= data_df['label']
print(X.head())
print('/n')
print(y.head())

0    biontech amp pfizer announce covid vaccine eff...
1    namanyaracheal ugandabreweries centurybottling...
2    minofhealthug receive total million dose covid...
3                              case think pfizer worst
4    sunday get vaccine usmissionuganda abt million...
Name: clean_text, dtype: object
/n
0    positive
1    positive
2     neutral
3    negative
4     neutral
Name: label, dtype: object


In [19]:
activity_count = y.value_counts()
activity_count

neutral     5459
positive    4429
negative    2648
Name: label, dtype: int64

In [20]:
# Splitting our data into train and test
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [21]:
# checking our classes
data_df.label.unique()

array(['positive', 'neutral', 'negative'], dtype=object)

In [22]:
# creating a list varaiable for our classes
target = ['negative','neutral','positive']

In [23]:
# Defining our X and Y varaibles
X_test = data_df['clean_text'].astype(str)
y_test = data_df['label'].astype(str)
print(X_test.head())
print('/n')
print(y_test.head())

0    biontech amp pfizer announce covid vaccine eff...
1    namanyaracheal ugandabreweries centurybottling...
2    minofhealthug receive total million dose covid...
3                              case think pfizer worst
4    sunday get vaccine usmissionuganda abt million...
Name: clean_text, dtype: object
/n
0    positive
1    positive
2     neutral
3    negative
4     neutral
Name: label, dtype: object


In [24]:
# converting our train varaibles to numpy array
X_train= X_train.to_numpy()
y_train= y_train.to_numpy()

In [25]:
# converting our test varaibles to numpy array
X_test= X_test.to_numpy()
y_test = y_test.to_numpy()

In [26]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
  
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased")

Downloading:   0%|          | 0.00/268M [00:00<?, ?B/s]

In [27]:
# defining our model object and preprocessing our data
MODEL_NAME = "distilbert-base-uncased"
t = text.Transformer(MODEL_NAME, maxlen=500, class_names= target)
trn = t.preprocess_train(X_train, y_train)
val = t.preprocess_test(X_test, y_test)
model = t.get_classifier()
learner = ktrain.get_learner(model, train_data=trn, val_data=val, batch_size=6)


preprocessing train...
language: en
train sequence lengths:
	mean : 10
	95percentile : 14
	99percentile : 15


/usr/local/lib/python3.7/dist-packages/ktrain/utils.py:621: UserWarning: class_names argument was ignored, as they were extracted from string labels in dataset
  if self.get_classes(): warnings.warn('class_names argument was ignored, as they were extracted from string labels in dataset')


Is Multi-Label? False
preprocessing test...
language: en
test sequence lengths:
	mean : 10
	95percentile : 14
	99percentile : 15


Downloading:   0%|          | 0.00/363M [00:00<?, ?B/s]

In [28]:
# Import required model class
from transformers import TFDistilBertForSequenceClassification
# Download pre-trained model
model = TFDistilBertForSequenceClassification.from_pretrained(MODEL_NAME, 
                                                              num_labels=3,
                                                              output_hidden_states=True, 
                                                              output_attentions=True
                                                              )

In [29]:
from transformers import DistilBertConfig
config = DistilBertConfig(n_heads=8, dim=512, hidden_dim=4*512)
model = TFDistilBertForSequenceClassification(config)

In [30]:
#initialise wandb
wandb.init(project='Covid vaccine sentiment', save_code= True, entity='Alpha-coders')



wandb: Currently logged in as: noahkandie (use `wandb login --relogin` to force relogin)


In [31]:
from transformers import TFTrainer,TFTrainingArguments
training_args = TFTrainingArguments(
    output_dir='./results',          
    num_train_epochs=3,              
    per_device_train_batch_size=16,  
    per_device_eval_batch_size=64,   
    warmup_steps=500,                
    weight_decay=0.01,
    # load_best_model_at_end=False,
    logging_dir='./logs',            
    logging_steps=10,
    report_to="wandb",  # enable logging to W&B
    run_name="Augmented-trainset"
       
)
with training_args.strategy.scope():
    model = TFDistilBertForSequenceClassification.from_pretrained("distilbert-base-uncased")
    


trainer = TFTrainer(
    model=model,                         
    args=training_args,                  
    train_dataset=trn,         
    eval_dataset=val             
)

In [32]:
learner.fit_onecycle(1e-5, 5, callbacks=[WandbCallback()])



begin training using onecycle policy with max lr of 1e-05...
Epoch 1/5
1672/1672 [==============================] - ETA: 0s - loss: 0.8614 - accuracy: 0.6001

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


1672/1672 [==============================] - 1153s 679ms/step - loss: 0.8614 - accuracy: 0.6001 - val_loss: 0.4709 - val_accuracy: 0.8315
Epoch 2/5
1672/1672 [==============================] - 1138s 679ms/step - loss: 0.3984 - accuracy: 0.8584 - val_loss: 0.2274 - val_accuracy: 0.9304
Epoch 3/5
1672/1672 [==============================] - 1138s 679ms/step - loss: 0.2286 - accuracy: 0.9271 - val_loss: 0.1273 - val_accuracy: 0.9643
Epoch 4/5
1672/1672 [==============================] - 1137s 678ms/step - loss: 0.1175 - accuracy: 0.9668 - val_loss: 0.0902 - val_accuracy: 0.9770
Epoch 5/5
1672/1672 [==============================] - 1137s 678ms/step - loss: 0.0694 - accuracy: 0.9824 - val_loss: 0.0801 - val_accuracy: 0.9803


In [33]:
from tensorflow.keras.models import load_model

model.save('Aug_model')

/usr/local/lib/python3.7/dist-packages/keras/saving/saved_model/layer_serialization.py:112: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  return generic_utils.serialize_keras_object(obj)


In [34]:
model = wandb.Artifact('learner', type='model')
model.add_file('Aug_model')

run.log_artifact(model)


ValueError: ignored

In [35]:
# predictor.save('Model')

NameError: ignored

In [36]:
# predictor = ktrain.load_predictor('/tmp/toxic_detector')

Exception: ignored

In [37]:
wandb.finish

<function wandb.sdk.wandb_run.finish>

In [38]:
def finish(exit_code: int = None) -> None:
    """Marks a run as finished, and finishes uploading all data.
    This is used when creating multiple runs in the same process.
    We automatically call this method when your script exits.
    """
    if wandb.run:
        wandb.run.finish(exit_code=exit_code)

In [39]:
learner.validate(class_names=t.get_classes())

              precision    recall  f1-score   support

    negative       0.97      0.97      0.97      2648
     neutral       0.99      0.98      0.99      5459
    positive       0.98      0.98      0.98      4429

    accuracy                           0.98     12536
   macro avg       0.98      0.98      0.98     12536
weighted avg       0.98      0.98      0.98     12536



array([[2578,   20,   50],
       [  40, 5371,   48],
       [  35,   54, 4340]])

In [40]:
# the one we got most wrong
learner.view_top_losses(n=1, preproc=t)

----------
id:2836 | loss:6.6 | true:negative | pred:neutral)



In [41]:
print(X_test[468])

receive covid vaccine people buikwe consume pfizer dose sinovac


In [42]:
predictor = ktrain.get_predictor(learner.model, preproc=t)

In [43]:
predictor.predict('would want people around depressed pearlradioke worldsuicidepreventionday cc')

'negative'

In [52]:
# Calculate Recall score
from sklearn import metrics
import numpy as np
y_pred=predictor.predict(X_test)
print(metrics.precision_score(y_test,y_pred,average='weighted'))

0.9803086966565688


In [53]:
preds= predictor.predict_proba('I’ve personally done bilateral above knee amputations for three patients due to COVID')
preds

array([0.00251004, 0.9933236 , 0.0041664 ], dtype=float32)

In [54]:
print([target], preds)
list_arrays= ([target], preds)

[['negative', 'neutral', 'positive']] [0.00251004 0.9933236  0.0041664 ]


In [55]:
pd.DataFrame(list(map(np.ravel, list_arrays)))

,0,1,2
0,negative,neutral,positive
1,0.00251004,0.993324,0.0041664


In [79]:
preds2= predictor.predict_proba('Im still not taking it. Why take a vaccine thats cleared for emergency use?')
preds2

array([0.5232345 , 0.43636495, 0.04040057], dtype=float32)

In [80]:
print([target], preds2)
list_arrays2= ([target], preds2)

[['negative', 'neutral', 'positive']] [0.5232345  0.43636495 0.04040057]


In [81]:
pd.DataFrame(list(map(np.ravel, list_arrays2)))

,0,1,2
0,negative,neutral,positive
1,0.523234,0.436365,0.0404006


In [82]:
preds3= predictor.predict_proba('vaccines are effective. They can keep you from getting and spreading the virus that causes COVID-19')
preds3

array([0.00137491, 0.0057945 , 0.9928306 ], dtype=float32)

In [83]:
print([target], preds3)
list_arrays3= ([target], preds3)

[['negative', 'neutral', 'positive']] [0.00137491 0.0057945  0.9928306 ]


In [84]:
pd.DataFrame(list(map(np.ravel, list_arrays3)))

,0,1,2
0,negative,neutral,positive
1,0.00137491,0.0057945,0.992831
